In [9]:

import requests
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import date,datetime,timedelta
import psycopg2
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import openpyxl
import xlrd
from dateutil.relativedelta import relativedelta
from dcon import PROJ_TRADING, HMBR_ID, HMBR_LOCAL_SERVER,  IT_MAIL, DIRECTOR_MAIL, MOTIUR_SIR_MAIL, SHAHALAM_MAIL
from mail import send_mail
import sys
import time

ENGINE = HMBR_LOCAL_SERVER

"""
        we need 

        last two month's before due of customer 
        suppose if current month is June then last 2 
        months ago will be April and date will be 30 April

        last months total sales amount of customer
        last months total collection from customer
        last months total return amount of customer

        current date till now sales
        current date till now return
        current date till now collection

        current date balance.

        At first we need to find out last two months ago's date, last months date and current month's date
        and put in the seperate variable
"""
check_today_is_friday = datetime.today().strftime("%A")
print(check_today_is_friday)

if check_today_is_friday == 'Friday':
    sys.exit()

#  Actual time delta will be one days before. As per direction of Bijoy Sir we reduce the period.
two_days_ago = datetime.now() - timedelta (1)
check_friday = two_days_ago.strftime("%A")

if check_friday == 'Friday':
    two_days_ago = datetime.now() - timedelta (2)

two_days_ago = two_days_ago.strftime("%Y-%m-%d")

### get two month's before last date
two_months_before_date  = (datetime.now() - timedelta(days=30)).replace(day=1) - timedelta(days=1)
two_months_before_date  = two_months_before_date.strftime('%Y-%m-%d')
### get last month's last date
last_month_date         = (datetime.now().replace(day=1) - timedelta(days=1)).strftime('%Y-%m-%d')
### get current date
current_month_date      = (datetime.now().strftime('%Y-%m-%d'))
print (current_month_date, two_months_before_date, last_month_date , end= " /")
#
def get_all_customer (zid):
    df = pd.read_sql(f"""SELECT xcus, xshort, xtaxnum, xcity,xstate
                      from cacus where  zid = {zid} order by xcus
                     """,con=ENGINE)
    return df
#
df_get_all_customer = get_all_customer(100001)

def get_acc_info(zid, proj, till_date , voucher_like):
# this date is for get the month name for naming the column
    date = datetime.strptime(till_date, '%Y-%m-%d')
# here the logic is, if voucher is ob then it will consider as due,
#  if voucher is inop then sale and 
# if the voucher is srt then it consider as return amount 
# as well as the amount which is month_year column name should be change respectively the voucher like april_2023

# this date is caluculated for the first day of the given till_date variable for getting Sales and return
    last_month_date = till_date
    last_month_datetime = datetime.strptime(last_month_date, "%Y-%m-%d")
    first_day_of_month = last_month_datetime.replace(day=1).strftime("%Y-%m-%d")

#  if voucher == 'OB--%% then query will be gldetail.xvoucher not like 'OB--%%' and glheader.xdate <= GIVEN DATE
    if voucher_like == "OB":
        voucher_condition = f"(glheader.xvoucher NOT LIKE 'OB--%%')"
        month_year = f"till_{date.strftime('%B_%Y')}_due"
        glheader_xdate = f"<= '{till_date}' "
        print (glheader_xdate)
#  if voucher == 'INOP%% then query will be gldetail.xvoucher like 'INOP%%' and glheader.xdate between 'From first day' and 'To given date'
    if voucher_like == "INOP":
        voucher_condition = f"(glheader.xvoucher LIKE 'INOP%%')"
        month_year = f"{date.strftime('%B_%Y')}_sale"
        glheader_xdate = f"<= '{till_date}' "
        print (glheader_xdate)

#  if voucher == 'SRT-%% then query will be gldetail.xvoucher like 'SRT-%%' and glheader.xdate between 'From first day' and 'To given date'
    if voucher_like == "SRT":
        voucher_condition = f"(glheader.xvoucher  LIKE 'SRJV%%' OR glheader.xvoucher LIKE 'SRT-%%')"
        month_year = f"{date.strftime('%B_%Y')}_return"
        glheader_xdate =f"<= '{till_date}' "
        print (glheader_xdate)
    if voucher_like == "credit_collection":
        voucher_condition = f"""(
            glheader.xvoucher  LIKE 'JV--%%' OR 
            glheader.xvoucher LIKE 'RCT-%%' OR
            glheader.xvoucher LIKE 'CRCT%%' OR
            glheader.xvoucher LIKE 'STJV%%' OR
            glheader.xvoucher LIKE 'BRCT%%' 
            )"""
        month_year = f"{date.strftime('%B_%Y')}_receive"
        
        glheader_xdate =f" <='{till_date}' "
        print (glheader_xdate)



    df = pd.read_sql(f"""SELECT gldetail.xsub as xcus,SUM(gldetail.xprime) as {month_year}
                        FROM glheader
                        JOIN gldetail
                        ON glheader.xvoucher = gldetail.xvoucher
                        JOIN cacus
                        ON gldetail.xsub = cacus.xcus
                        WHERE glheader.zid = {zid}
                        AND gldetail.zid = {zid}
                        AND cacus.zid = {zid}
                        AND gldetail.xproj = '{proj}'
                        AND {voucher_condition}
                        AND glheader.xdate {glheader_xdate}
                        GROUP BY gldetail.xsub""",con=ENGINE)
    return df

#
pd.set_option('display.float_format', '{:.2f}'.format)
#
# now current month e.g june month sale from day 01 to till date
current_month_sale_df = get_acc_info(HMBR_ID, PROJ_TRADING, current_month_date, 'INOP')
# add to previous dataframe and merge

#
# june or current month return amount srt voucher
current_month_return_df = get_acc_info(HMBR_ID, PROJ_TRADING, current_month_date, 'SRT')
# add to previous dataframe and merge
current_month_return_df = pd.merge(current_month_sale_df, current_month_return_df, on='xcus', how='left')
current_month_return_df.head(2)


#
# june or current month credit collection
current_month_collection_df = get_acc_info(HMBR_ID, PROJ_TRADING, current_month_date, 'credit_collection')
# add to previous dataframe and merge and fill all nan to 0
current_month_collection_df = pd.merge(current_month_return_df, current_month_collection_df, on='xcus', how='left').fillna(0)
# fill all negative value to positive absolute value
current_month_collection_df.iloc[:, 5:] = current_month_collection_df.iloc[:, 5:].abs()

current_month_balance_df = current_month_collection_df


current_month_balance_df

Monday
2023-12-04 2023-10-31 2023-11-30 /<= '2023-12-04' 
<= '2023-12-04' 
 <='2023-12-04' 


,xcus,december_2023_sale,december_2023_return,december_2023_receive
0,CUS-009105,159934.00,-5120.00,-154814.00
1,CUS-000033,718301.95,-75546.00,-642755.95
2,CUS-003820,11050.00,-7385.00,-3665.00
3,CUS-004900,51538.00,-4607.00,-46931.00
4,CUS-008752,119008.00,-18181.00,-100827.00
...,...,...,...,...
9191,CUS-007840,14778.00,-4050.00,-10728.00
9192,CUS-004209,345898.40,-83048.92,-262850.40
9193,CUS-009266,84587.00,-8192.00,-76395.00
9194,CUS-000482,67874.00,-20449.00,-47425.00


In [6]:

# get till date balance
current_month_balance_df['balance'] = current_month_balance_df.apply(lambda row: (row.iloc[5] + row.iloc[6] + row.iloc[9]) - (row.iloc[7] + row.iloc[8] + row.iloc[10] + row.iloc[11]), axis=1)

current_month_balance_df.head(2)

#
# now merge with previous dataframe last_month_total_credit_collection_df
last_month_name_balance = datetime.strptime(last_month_date, '%Y-%m-%d').strftime('%B_%Y_balance') 
current_month_name_balance = datetime.strptime(current_month_date, '%Y-%m-%d').strftime('%B_%Y_balance')

current_month_name_balance

,xcus,xshort,xtaxnum,xcity,xstate,till_october_2023_due,november_2023_sale,november_2023_receive,november_2023_return,december_2023_sale,december_2023_return,december_2023_receive,balance
0,CUS-000001,HMBR FiX iT,,Gulshan-1,Dhaka retail,938011.48,110858.00,250000.00,2678.00,0.00,0.00,0.00,796191.48
1,CUS-000002,Gulshan Hardware,,Gulshan-1,Dhaka retail,125477.44,0.00,0.00,0.00,0.00,0.00,0.00,125477.44
2,CUS-000003,Rahima Enterprise,,Comilla,Rahima enterprise,5248413.47,2532910.00,1605995.00,104415.30,66720.00,0.00,0.00,6137633.17
3,CUS-000004,Gulshan Trading Company,8801711881978,Vatara,Dhaka retail,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,CUS-000005,Gulshan Thread Tape,8801711881978,Vatara,Dhaka retail,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9880,CUS-009996,Goldmine Hardware & Electric,8801608700313,Damra,Dhaka retail,2290.22,15730.00,12340.00,5680.00,0.00,0.00,0.00,0.22
9881,CUS-009997,Enan Hardware Sanitary & Electric,8801920997433,Damra,Dhaka retail,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9882,CUS-009998,Syed Hardware & Electric,8801624940033,Saver,Dhaka retail,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9883,CUS-009999,Najmul hardware,8801795649425,Habiganj,Sylhet Retail,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
